Authors:

- Kacper Kuźnik 75267
- Mikołaj Nowacki 75231

In [ ]:
%pip install --upgrade torch torchvision torchaudio transformers ultralytics torchmetrics pycocotools timm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.7/899.7 MB 1.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 103.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 MB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 954.8/954.8 kB 55.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.1/193.1 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 54.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.6/63.6 MB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.5/267.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.2/288.2 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.3/39.3 MB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90

## Mount Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


## Load Fine-tuned Models

In [ ]:
import os
import random
import torch
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image
from torch.utils.data import Dataset
from transformers import AutoImageProcessor, AutoModelForObjectDetection

# --- CONFIGURATION ---
CHECKPOINT_PATH = "/content/drive/MyDrive/CV-project/rtdetr-v2-r50-taco-finetune/checkpoint-1000"

# Path to the UAVVaste subset images
UAV_IMG_DIR = "/content/drive/MyDrive/CV-project/UAVWasteDataset_subset"

# Confidence threshold for visualization
CONF_THRESHOLD = 0.25

In [ ]:
# --- 1. DATASET LOADER ---
class UAVImageFolder(Dataset):
    def __init__(self, img_dir):
        self.img_dir = img_dir
        # valid extensions
        self.images = [f for f in sorted(os.listdir(img_dir))
                       if f.lower().endswith(('.jpg', '.jpeg', '.png'))]

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        file_name = self.images[idx]
        img_path = os.path.join(self.img_dir, file_name)
        try:
            image = Image.open(img_path).convert("RGB")
            return image, file_name
        except Exception as e:
            print(f"Error loading {file_name}: {e}")
            return None, None

In [ ]:
# --- 2. LOAD MODEL ---
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Loading model from: {CHECKPOINT_PATH}")
print(f"Using device: {device}")

try:
    processor = AutoImageProcessor.from_pretrained(CHECKPOINT_PATH)
    model = AutoModelForObjectDetection.from_pretrained(CHECKPOINT_PATH).to(device)

    id2label = getattr(model.config, "id2label", {})
except Exception as e:
    print(f"Error loading model. Ensure the path is correct and it is a Hugging Face checkpoint.\nError: {e}")

Loading model from: /content/drive/MyDrive/CV-project/rtdetr-v2-r50-taco-finetune/checkpoint-1000
Using device: cpu


In [ ]:
# --- 2,5. LOAD PREVIOUS MODELS ---
# from ultralytics import YOLO
from PIL import Image
from transformers import RTDetrForObjectDetection, RTDetrImageProcessor, DetrForObjectDetection, DetrImageProcessor
import torch

#YOLO
# yolo_model = YOLO('yolo11n.pt')

#DETR
detr_model = DetrForObjectDetection.from_pretrained("facebook/detr-resnet-50")
detr_processor = DetrImageProcessor.from_pretrained("facebook/detr-resnet-50")
device = "cuda" if torch.cuda.is_available() else "cpu"
detr_model.to(device)
detr_model.eval()


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/167M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/102M [00:00<?, ?B/s]

/usr/local/lib/python3.12/dist-packages/torch/nn/modules/module.py:2446: UserWarning: for conv1.weight: copying from a non-meta parameter in the checkpoint to a meta parameter in the current model, which is a no-op. (Did you mean to pass `assign=True` to assign items in the state dictionary to their corresponding key in the module instead of copying them in place?)
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torch/nn/modules/module.py:2446: UserWarning: for bn1.weight: copying from a non-meta parameter in the checkpoint to a meta parameter in the current model, which is a no-op. (Did you mean to pass `assign=True` to assign items in the state dictionary to their corresponding key in the module instead of copying them in place?)
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torch/nn/modules/module.py:2446: UserWarning: for bn1.bias: copying from a non-meta parameter in the checkpoint to a meta parameter in the current model, which is a no-op. (Did you mean to pas

preprocessor_config.json:   0%|          | 0.00/290 [00:00<?, ?B/s]

DetrForObjectDetection(
  (model): DetrModel(
    (backbone): DetrConvModel(
      (conv_encoder): DetrConvEncoder(
        (model): FeatureListNet(
          (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
          (bn1): DetrFrozenBatchNorm2d()
          (act1): ReLU(inplace=True)
          (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
          (layer1): Sequential(
            (0): Bottleneck(
              (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
              (bn1): DetrFrozenBatchNorm2d()
              (act1): ReLU(inplace=True)
              (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
              (bn2): DetrFrozenBatchNorm2d()
              (drop_block): Identity()
              (act2): ReLU(inplace=True)
              (aa): Identity()
              (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      

In [ ]:
# --- 3. INFERENCE & VISUALIZATION FUNCTION ---
def visualize_predictions(dataset, model, processor, indices, threshold=0.3):

    # Select random images
    for i in indices:
        image, fname = dataset[i]
        if image is None: continue

        # Preprocess
        inputs = processor(images=image, return_tensors="pt").to(device)

        # Inference
        with torch.no_grad():
            outputs = model(**inputs)

        # Post-process (convert raw outputs to boxes)
        target_sizes = torch.tensor([image.size[::-1]]).to(device)
        results = processor.post_process_object_detection(
            outputs, target_sizes=target_sizes, threshold=threshold
        )[0]

        boxes = results["boxes"].cpu()
        scores = results["scores"].cpu()
        labels = results["labels"].cpu()

        # Visualization
        plt.figure(figsize=(10, 10))
        plt.imshow(image)
        ax = plt.gca()

        # If no detections
        if len(boxes) == 0:
            plt.title(f"{fname}\nNo detections (thresh={threshold})")
            plt.axis('off')
            plt.show()
            continue

        # Draw boxes
        for box, score, label in zip(boxes, scores, labels):
            x1, y1, x2, y2 = box.tolist()
            w, h = x2 - x1, y2 - y1

            # Label text
            label_name = id2label.get(label.item(), str(label.item()))
            display_text = f"{label_name}: {score:.2f}"

            # Draw rectangle
            rect = patches.Rectangle((x1, y1), w, h, linewidth=2, edgecolor='red', facecolor='none')
            ax.add_patch(rect)

            # Draw label background
            ax.text(x1, y1, display_text, color='white', fontsize=10,
                    bbox=dict(facecolor='red', alpha=0.5))

        plt.title(f"Image: {fname} | Detected: {len(boxes)}")
        plt.axis('off')
        plt.show()

## Initialize Dataset

In [ ]:
# Initialize dataset
uav_dataset = UAVImageFolder(UAV_IMG_DIR)

In [ ]:
CONF_THRESHOLD = 0.1

In [ ]:
# Get indices for random images
num_images = 5
rand_indices = random.sample(range(len(uav_dataset)), min(num_images, len(uav_dataset)))

## Run the fine-tuned model

In [ ]:
# Run visualization
visualize_predictions(uav_dataset, model, processor, rand_indices, threshold=CONF_THRESHOLD)

Output hidden; open in https://colab.research.google.com to view.

## Run the pre-trained model

In [ ]:
visualize_predictions(uav_dataset, detr_model, detr_processor, rand_indices, threshold=CONF_THRESHOLD)

Output hidden; open in https://colab.research.google.com to view.